# Safe and Just Earth Systems Boundaries for Surface Water: Hydrologic Alteration of Environmental Flows

***
<font color=red>Title: Safe and Just Earth Systems Boundaries for Surface Water: Hydrologic Alteration Environmental Flows  
Author: Pamela A. Green  
Date: February 17, 2023  

This code was developed between December, 2021 and February, 2023 by Pamela Green, Senior Research Associate, CUNY Advanced Science Research Center, New York, NY under subcontract to Griffith University.  </font>
***

This code represents spatial modelling for development of the safe and just surface water target for Working Group 3 of the Earth Commission for the Earth Commission Long Report and the <i><b>"Safe and Just Earth Systems Boundaries"</b></i> publication. The surface water target includes spatial modelling of the extent of global-scale hydrological alteration of environmental flows.

Being outside the Earth System Boundary is based on the concept of environmental flow stress; defined as an alteration of surface water flow between pristine (non-human impacted) and disturbed (human impacted) conditions that exceed a given threshold. We derived the pristine and disturbed monthly river flow datasets from the WBM water balance model river discharge outputs (Wisser et al. 2010) at 6-minute grid cell resolution using the TerraClimate high resolution data set of monthly climate forcings (Abatzoglou, et al. 2018) for the period 2000-2020. River basin delineation and flow routing configurations are defined by the WBM 6-minute topological river network used to establish local discharge and river flow (Wisser et al. 2010). The pristine and disturbed WBM runs use the same climate forcings for the 2000-2020 time period but only employ human alterations to the water cycle, including water extraction for irrigation and large reservoirs, in the disturbed runs. 

Long-term mean monthly discharge is calculated for the modelled pristine (non-human impacted) and disturbed (human impacted) discharge from the WBM model over the 2000-2020 time domain to determine the extent of altered flow. The analysis is limited to only the perennial or actively flowing river extents by applying a 3mm/yr upstream monthly average runoff exceedance threshold (Fekete et al. 2001) occurring for at least 10 years out of the 2000-2020 time domain. We also mask out upstream headwater areas (smaller than 250km2) that have modelled irrigation depths below the median irrigation depth for small headwater cells (3.6 mm/yr). This mask is applied to eliminate noise in the modelled data associated with very low irrigation and discharge values in headwater grid cells.

This work is part of the Earth Commission which is hosted by Future Earth and is the science component of the Global Commons Alliance. The Global Commons Alliance is a sponsored project of Rockefeller Philanthropy Advisors, with support from Oak Foundation, MAVA, Porticus, Gordon and Betty Moore Foundation, Herlin Foundation and the Global Environment Facility. The Earth Commission is also supported by the Global Challenges Foundation. 

<b>References:</b>

Abatzoglou, John T., Solomon Z. Dobrowski, Sean A. Parks, and Katherine C. Hegewisch. “TerraClimate, a High-Resolution Global Dataset of Monthly Climate and Climatic Water Balance from 1958–2015.” Scientific Data 5, no. 1 (January 9, 2018): 170191. https://doi.org/10.1038/sdata.2017.191.

Center for International Earth Science Information Network - CIESIN - Columbia University. 2018. Gridded Population of the World, Version 4 (GPWv4): Population Density, Revision 11. Palisades, NY: NASA Socioeconomic Data and Applications Center (SEDAC). https://doi.org/10.7927/H49C6VHW.

Fekete, Balázs M., Charles J. Vörösmarty, and Richard B. Lammers. “Scaling Gridded River Networks for Macroscale Hydrology: Development, Analysis, and Control of Error.” Water Resources Research 37, no. 7 (July 2001): 1955–67. https://doi.org/10.1029/2001WR900024.

GRDC (2020): Major River Basins of the World / Global Runoff Data Centre, GRDC. 2nd, rev. ext. ed. Koblenz, Germany: Federal Institute of Hydrology (BfG).

Wisser, D., B. M. Fekete, C. J. Vörösmarty, and A. H. Schumann. “Reconstructing 20th Century Global Hydrography: A Contribution to the Global Terrestrial Network- Hydrology (GTN-H).” Hydrology and Earth System Sciences 14, no. 1 (January 6, 2010): 1–24. https://doi.org/10.5194/hess-14-1-2010.


# Modules and Functions

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy

from scipy import stats
from osgeo import gdal, gdal_array, osr, ogr
from osgeo.gdalconst import *
import xarray as xr

from math import floor

## Single band numpy array to GeoTiff file

In [ ]:
def save2File(rA, outname, nrows, ncols, geo_transform):  # Resolution for the 6min
    # create the output image
    # Note that in the geo transform the third and sixth parameters are equal to the Arc/Info generate fishnet
    # Y-coordinate paramenter (defining the rotation of the final grid)
    # LCORD
    outDs = gdal.GetDriverByName('GTiff').Create(outname, ncols, nrows, 1, gdal.GDT_Float32)
    outBand = outDs.GetRasterBand(1)
    outBand.WriteArray(rA)
    outDs.SetGeoTransform(geo_transform)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)
    outDs.SetProjection(srs.ExportToWkt())
    outDs = None

    """
    geotransform[0] = top left x
    geotransform[1] = w-e pixel resolution
    geotransform[2] = 0
    geotransform[3] = top left y
    geotransform[4] = 0
    geotransform[5] = n-s pixel resolution (negative value)
    """

## MultiBand numpy array to GeoTiff

In [ ]:
def CreateMultiGeoTiff(Array, Name, driver, NDV, GeoT, Projection, DataType):
    Array[np.isnan(Array)] = NDV
    DataSet = gdal.GetDriverByName(driver).Create(Name, Array.shape[2], Array.shape[1], Array.shape[0], DataType)
    DataSet.SetGeoTransform(GeoT)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(Projection)
    DataSet.SetProjection(srs.ExportToWkt() )
    for i, image in enumerate(Array, 1):
        DataSet.GetRasterBand(i).WriteArray( image )
        DataSet.GetRasterBand(i).SetNoDataValue(NDV)
    DataSet.FlushCache()
    return Name

# Global Variables, Definitions, Datasets

## Global Pathways and Data files

In [ ]:
main = os.getcwd( ) + '/'

in_dir = str(main) + 'ModelInput/'
out_dir = str(main) + 'ModelOutput/'

## Explicitely define projection information for 6 minute rasters

In [ ]:
n = 300

ncols6m = 3600
nrows6m = 1500
xll6m = -180
yll6m = -60
xur6m = 180
yur6m = 90
cellsize6m = 0.1
cellang = cellsize6m * -1
nodata = -9999
geotr6m = ([ xll6m, cellsize6m, 0, yur6m, 0, cellang ])

geodriver = 'GTiff'
geoproj = 4326

reso = '6min'

head6m = 'ncols ' + str(ncols6m) + '\nnrows ' + str(nrows6m) + '\nxllcorner ' + str(xll6m) + '\nyllcorner ' + str(yll6m) + '\ncellsize ' + str(cellsize6m) + '\nNODATA_value ' + str(nodata)

## Read in RGIS GHAAS3 Basin and Subbasin, Grid Cellarea, Uptsream Area, and Flow Mask rasters at 6min resolution

In [ ]:
fin = str(in_dir) + 'RiverBasin/HydroSTN06_Subbasin.tif'
ds = gdal.Open(str(fin))
subbas = np.array(ds.GetRasterBand(1).ReadAsArray())

del ds

fin = str(in_dir) + 'RiverBasin/Global_Basin_HydroSTN30_06min_GHAAS3.tif'
ds = gdal.Open(str(fin))
basin = np.array(ds.GetRasterBand(1).ReadAsArray())
basin[np.isnan(basin)] = 0

basin_name_file = 'MajorBasinLookUp.xlsx'
pd_basin_name = pd.read_excel(in_dir + 'RiverBasin/' + basin_name_file, sheet_name='LookupVALUES')
del pd_basin_name['BasinName']
del pd_basin_name['CONTINENT']
del pd_basin_name['SEA']
del pd_basin_name['OCEAN']
del pd_basin_name['TFDD']
pd_basin_name = pd_basin_name[(pd_basin_name['ID'] != 0)]

fin = str(in_dir) + 'cellarea/Global_UpstreamAreakm2_6min.tif'
ds = gdal.Open(str(fin))
UpArea = np.array(ds.GetRasterBand(1).ReadAsArray())
UpArea[np.isnan(UpArea)] = 0

fin = str(in_dir) + 'cellarea/CellArea_6m.tif'
ds = gdal.Open(str(fin))
CellArea = np.array(ds.GetRasterBand(1).ReadAsArray())
CellArea[np.isnan(CellArea)] = 0

sum_basin_area = scipy.ndimage.sum(CellArea, basin, basin)
sum_subbasin_area = scipy.ndimage.sum(CellArea, subbas, subbas)

# Actively flowing mask areas are based on Upstream Runoff less than 3mm occuring 10 or more times 
# in any given year over the time period of study (2000-2021). Problematic low- and non-flow basins 
# removed due to WBM difficulties in dry regions
fin = in_dir + 'WBM_data/flow_mask.tif'
ds = gdal.Open(str(fin))
flow_mask = np.array(ds.GetRasterBand(1).ReadAsArray())
flow_mask[np.isnan(flow_mask)] = 0

# Import Water Balance Model and Population Datasets

## Read in WBM-TerraClimate Long-Term Mean (LTM) Raster Discharge Data

Convert to km3/yr

Calculate Available Surface Water volume = Pristine LTM Discharge * 0.2

In [ ]:
print("Read in LTM WBM data..")

# Saved discharge from earlier - newer discharge is unsurprisingly different.

savedQ_dir = str(in_dir) + 'WBM_data/'

filepath = savedQ_dir + 'WBM_TerraClimate_OUTPUTQ_PRIST_aLTM_2000-2020.tif'
dataset = gdal.Open(filepath)
WBMpristQ_ann = dataset.ReadAsArray()

filepath = savedQ_dir + 'WBM_TerraClimate_OUTPUTQ_DIST_aLTM_2000-2020.tif'
dataset = gdal.Open(filepath)
WBMdistQ_ann = dataset.ReadAsArray()

filepath = savedQ_dir + 'WBM_TerraClimate_OUTPUTQ_PRIST_mLTM_2000-2020.tif'
dataset = gdal.Open(filepath)
WBMpristQ_mo = dataset.ReadAsArray()
    
filepath = savedQ_dir + 'WBM_TerraClimate_OUTPUTQ_DIST_mLTM_2000-2020.tif'
dataset = gdal.Open(filepath)
WBMdistQ_mo = dataset.ReadAsArray()

diff = WBMpristQ_mo - WBMdistQ_mo
absdiff = np.absolute(diff)
absfrac = np.where(absdiff[:, :] > 0.0000000, np.where(WBMpristQ_mo == 0.0, 1, absdiff[:, :] / WBMpristQ_mo), 0.0)   

##########################################
#Discharges in km3/yr
##########################################

WBMpristQ_ann_km3yr = WBMpristQ_ann  * 0.031558464
WBMdistQ_ann_km3yr = WBMdistQ_ann  * 0.031558464    
availWBMpristQ_ann_km3yr = WBMpristQ_ann_km3yr * 0.2

## Import WBM-TerraClimate Net Irrigation Water Demand LTM and Identify quantiles for headwater cells


In [ ]:
filepath = in_dir + 'WBM_data/WBM_TerraClimate_OUTPUT_IrrNetWD_aLTM_2000-2020.tif'
dataset = gdal.Open(filepath)
irrrovol_mean = dataset.ReadAsArray()

irrromm_mean = irrrovol_mean / CellArea / 0.0000316880878

UpAreaLim = 250
irrromm_headw = np.where(UpArea > UpAreaLim, irrromm_mean, 0)

irrmm_flat = irrromm_headw.reshape(-1)
df_mm = pd.DataFrame(irrmm_flat, columns = ['IrrNetmm'])
df_mm = df_mm.dropna()
df_mm_flat = df_mm.loc[~(df_mm == 0).any(axis=1)]

outname = in_dir + 'WBM_data/WBM_TerraClimate_OUTPUT_IrrNetWD_mm_aLTM_2000-2020.tif'
save2File(irrromm_mean[:, :], outname, nrows6m, ncols6m, geotr6m)

## Read in Raster Population

Population datasets are from CIESIN GPW4(references below) and were resampled to 6min resolution.

Contemporary Population:
- Center for International Earth Science Information Network - CIESIN - Columbia University. 2018. Gridded Population of the World, Version 4 (GPWv4): Population Density, Revision 11. Palisades, NY: NASA Socioeconomic Data and Applications Center (SEDAC).


In [ ]:
fin = str(in_dir) + 'population/gpw_v4_population_count_rev11_2020_6min.tif'

ds = gdal.Open(str(fin))
pop2020 = np.array(ds.GetRasterBand(1).ReadAsArray())
pop2020[np.isnan(pop2020)] = 0
pop2020 = pop2020[ :-n, :]

sum_pop2020 = scipy.ndimage.sum(pop2020, subbas, subbas)
sum_basin_pop2020 = scipy.ndimage.sum(pop2020, basin, basin)

# Build Environmental Flow Rasters and Tables

Environmental stress is defined as an alteration of flow between pristine (non-human impacted) and disturbed (human impacted) conditions that exceed a given threshold. We derived the pristine and disturbed monthly river flow datasets from the WBM water balance model river discharge outputs (Wisser et al. 2010) at 6-minute grid cell resolution using the TerraClimate high resolution data set of monthly climate forcings (Abatzoglou, et al. 2018) for the period 2000-2020. Long-term mean monthly discharge is calculated for the modelled pristine (non-human impacted) and disturbed (human impacted) discharge from the WBM model over the 2000-2020 time domain to determine the extent of altered flow. 

To quantify the extent of alteration of surface flows, we first calculate the number of months in a year where the contemporary disturbed discharge is more than 20% different from pre-industrial discharge using the long-term mean gridded discharge data. We then represent this data as the proportion of months in a year with more than 20% difference (divide by 12 months/year). 

For final raster datasets we limit the extent of environmental stress to only the perennial or actively flowing river extents by applying a 3mm/yr upstream monthly average runoff exceedance threshold (Fekete et al. 2001) occurring for at least 10 years out of the 2000-2020 time domain. We also mask out upstream headwater areas (smaller than 250km2) that have modelled irrigation depths below the median irrigation depth for small headwater cells (3.6 mm/yr). This mask is applied to eliminate noise in the modelled data associated with very low irrigation and discharge values in headwater grid cells.

Finally, we calculate a basin-level estimate of the proportion of grid cells where discharge is more than 20% different by taking the basin-wide mean of the gridded proportion of months over the actively flowing river extents within the basin. We also calculate the population weighted average of this index using the GPw4 population data (CIESIN 2018) for 2020, and a discharge weighted average of the index using the long-term mean pre-industrial flows from our model runs. River network and basin extents are defined by the WBM water balance model with naming convention taken from the GRDC Major River Basins of the World (GRDC 2020).  

In [ ]:
chg_dir = 'estress_chg20'
chg_nm = 'Chg20'
chg = 0.2

##########################################
# ESTRESS = where the difference between Pristine and Disturbed Discharge (absfrac) exceeds 20% limit
# absfrac = abs(Pristine_LTM_Q - Disturbed_LTM_Q) / Pristine_LTM_Q 
# 6-minute raster data is created below then aggregated to basin level in following section 
##########################################

print("Create ESTRESS rasters from LTM for " + chg_dir)

estress = np.where(flow_mask == 1.0, 0.0, np.where(absfrac[:, :] >= chg, 1.0, 0.0))  
estressLTM = np.sum(estress, axis = 0)
estressLTM = np.where(flow_mask == 0, estressLTM, np.NaN)
estressLTM = estressLTM[:,:]

# Apply flow and min irrigation mask to eliminate noise in raster
UpAreaLim = 250
#75% = 0.1; 34.02mm
#50% = 0.01; 3.681mm
#25% = 0.001; 0.373mm
#10% = 0.0001; 0.04 mm
IrrLim = df_mm_flat.IrrNetmm.quantile(0.5)

estressLTM_raster = np.where(flow_mask == 0, np.where(UpArea > UpAreaLim, estressLTM, np.where(irrromm_mean > IrrLim, estressLTM, 0)), np.NaN)

nans = np.isnan(estressLTM_raster)
estressLTM_CNTnonans = np.where(nans, 0.0, 1.0)
estressLTM_nonans = np.where(nans, 0.0, estressLTM_raster)

outname = out_dir + 'Estress_WBM_TerraClimate_2000-2020_LTM_' + chg_nm + '.tif'
save2File(estressLTM_raster[:, :], outname, nrows6m, ncols6m, geotr6m)
# outname = out_dir + 'ModelOutput/' + chg_dir + '/Estress_WBM_TerraClimate_2000-2020_LTM_CNT' + chg_dir + '.tif'
# save2File(estressLTM_CNTnonans[:, :], outname, nrows6m, ncols6m, geotr6m)
# outname = out_dir + 'ModelOutput/' + chg_dir + '/Estress_WBM_TerraClimate_2000-2020_LTM_NONANS' + chg_dir + '.tif'
# save2File(estressLTM_nonans[:, :], outname, nrows6m, ncols6m, geotr6m)

print("Create ESTRESS Basin/Subbasin Data..")

###########################
# Aggregating 6-minute ESTRESS Metric to Basins
# Arithmentic Mean over Basin
###########################

estressLTM_CNTsum = scipy.ndimage.sum(estressLTM_CNTnonans, basin, basin)
estressLTM_sum = scipy.ndimage.sum(estressLTM_nonans, basin, basin)

estressLTM_basin = (estressLTM_sum / estressLTM_CNTsum) / 12

###########################
# Aggregating 6-minute ESTRESS Metric to Basins
# Pop Weighted Mean over Basin
#################################

estressLTM_NONULL = np.where(flow_mask == 1.0, 0.0, np.where(estressLTM >= 0, estressLTM, 0.0))
tempcal = estressLTM_NONULL * pop2020
basin_popwgtESTRESS = (scipy.ndimage.sum(tempcal, basin, basin) / sum_basin_pop2020 ) / 12

cnt = np.where(flow_mask == 1.0, 0.0, np.where(estressLTM >= 0, 1, 0))
basin_meanESTRESS = (scipy.ndimage.sum(estressLTM_NONULL, basin, basin) / scipy.ndimage.sum(cnt, basin, basin)) / 12

###########################
# Aggregating 6-minute ESTRESS Metric to Basins
# Discharge Weighted Mean over Basin 
#################################

tempcal = estressLTM_NONULL * WBMpristQ_ann_km3yr
sum_basin_Q = scipy.ndimage.sum(WBMpristQ_ann_km3yr, basin, basin)
basin_QwgtESTRESS = (scipy.ndimage.sum(tempcal, basin, basin) / sum_basin_Q ) / 12

## Link Data to Subbasin and Basin vector files by ID and output to spreadsheets

Subbasin and Basin IDs link to WBM basin shapefiles for map creation

In [ ]:
#################################
#Build Basin/Subbasin Tables
#################################

shp_filename = in_dir + 'RiverBasin/HydroSTN30_Confluence_6m.shp'
ds=ogr.Open(shp_filename)
lyr=ds.GetLayer()

li_values = list()
liincr_values = list()
lidecr_values = list()

for feat in lyr:
    geom = feat.GetGeometryRef()
    feat_id = feat.GetField('ID')
    mx, my = geom.GetX(), geom.GetY()
    px = floor((180 + mx) / 0.1)
    py = floor((90 - my) / 0.1)
    li_values.append([feat_id, estressLTM[py, px]])

df = pd.DataFrame(li_values)
filenmXL = out_dir + 'SUBBASIN_Stats_EFLOWS' + chg_nm + '.xlsx'  
with pd.ExcelWriter(filenmXL) as writer:
    df.to_excel(writer, index=False, sheet_name='estressLTM', header=['ID', 'estressLTM'])

shp_filename = in_dir + 'RiverBasin/HydroSTN06_BasinMouth.shp'
ds=ogr.Open(shp_filename)
lyr=ds.GetLayer()

values1 = list()
values2 = list()
values3 = list()
values4 = list()
values5 = list()
values6 = list()
values7 = list()
values8 = list()

for feat in lyr:
    geom = feat.GetGeometryRef()
    feat_id = feat.GetField('ID')
    mx, my = geom.GetX(), geom.GetY()
    px = floor((180 + mx) / 0.1)
    py = floor((90 - my) / 0.1)
    values1.append([feat_id, sum_basin_area[py, px]])
    values2.append([feat_id, WBMpristQ_ann_km3yr[py, px]])
    values3.append([feat_id, availWBMpristQ_ann_km3yr[py, px]])
    values4.append([feat_id, WBMdistQ_ann_km3yr[py, px]])
    values5.append([feat_id, sum_basin_pop2020[py, px]])
    values6.append([feat_id, estressLTM_basin[py, px]])
    values7.append([feat_id, basin_popwgtESTRESS[py, px]])
    values8.append([feat_id, basin_QwgtESTRESS[py, px]])

df1 = pd.DataFrame(values1)
df2 = pd.DataFrame(values2)
df3 = pd.DataFrame(values3)
df4 = pd.DataFrame(values4)
df5 = pd.DataFrame(values5)
df6 = pd.DataFrame(values6)
df7 = pd.DataFrame(values7)
df8 = pd.DataFrame(values8)

df1.columns =['ID', 'sum_basin_area']
df2.columns =['ID', 'WBMpristQ_ann_km3yr']
df3.columns =['ID', 'availWBMpristQ_ann_km3yr']
df4.columns =['ID', 'WBMdistQ_ann_km3yr']
df5.columns =['ID', 'sum_basin_pop2020']
df6.columns =['ID', 'estressLTM_basin']
df7.columns =['ID', 'basin_popwgtESTRESS']
df8.columns =['ID', 'basin_QwgtESTRESS']


df = pd_basin_name
df = pd.merge(df, df1, on=['ID'])
df = pd.merge(df, df2, on=['ID'])
df = pd.merge(df, df3, on=['ID'])
df = pd.merge(df, df4, on=['ID'])
df = pd.merge(df, df5, on=['ID'])
df = pd.merge(df, df6, on=['ID'])
df = pd.merge(df, df7, on=['ID'])
df = pd.merge(df, df8, on=['ID'])

filenmXL = out_dir + 'BASIN_Stats_EFLOWS' + chg_nm + '.xlsx'
with pd.ExcelWriter(filenmXL) as writer:
    df.to_excel(writer, index=False, sheet_name='Basin_Frac_ALL', header=['ID', 'Basin Name', 'Basin Area (km2)', 
                                                                          'Pristine Discharge (km3/yr)', 'Available Pristine Discharge (km3/yr)', 'Contemporary Discharge (km3/yr)', 
                                                                          'Total Population 2020', 'Proportion of Months where Flows Differ (Basin mean)', 'Proportion of Months where Flows Differ (Population weighted)', 'Proportion of Months where Flows Differ (Discharge weighted)'])
del df1
del df2
del df3
del df4
del df5
del df6
del df7
del df8
del df